In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import uuid
from langchain_core.messages import HumanMessage
from src.ai.autodf_ml_assistant import build_graph

In [ ]:
async def visualize_dataframe_assistant_graph():
    graph = await build_graph()
    
    return graph

In [ ]:
assistant_graph = await visualize_dataframe_assistant_graph()
print(assistant_graph.get_graph().draw_mermaid())

In [ ]:
assistant_graph

In [ ]:
thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id" : thread_id}}

In [ ]:
async def log_assistant_graph_steps(query: str, config: dict):
    step = 1
    current_tool = None
    async for token, metadata in assistant_graph.astream(
            {"messages": [HumanMessage(content=query)]}, config=config, stream_mode="messages"
    ):
        current_node = metadata.get('langgraph_node', 'unknown')
        
        # Check if this is a tool call by looking at the message content
        if hasattr(token, 'tool_calls') and token.tool_calls:
            for tool_call in token.tool_calls:
                tool_name = tool_call.get('name', 'unknown_tool')
                print(f"\n🔧 Calling tool: {tool_name}")
                current_tool = tool_name
        
        # Print step metadata when it changes
        if metadata['langgraph_step'] == step:
            print(f"\n🔄 Step {step}: Node '{current_node}'")
            if current_node == 'tools' and current_tool:
                print(f"   └── Executing tool: {current_tool}")
            print("Metadata", metadata, "\n")
            step += 1
            
        print(token.content, end='', flush=True)

In [ ]:
query = "List the columns of my data."
await log_assistant_graph_steps(query, config)

In [ ]:
query = "please give me descriptive statistics of the data"
await log_assistant_graph_steps(query, config)

In [ ]:
query = "Can we do predictive modeling on this dataset?"
await log_assistant_graph_steps(query, config)